In [1]:
!pip install torch tiktoken transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.7 MB/s eta 0:00:0000:01


# Tokenization and Embeddings for LLMs

This notebook covers the data preparation pipeline for training a GPT-like model:
1. **Tokenization** - Converting text to numbers (token IDs)
2. **Creating training data** - Sliding window approach for next-token prediction
3. **Token embeddings** - Converting token IDs to dense vectors
4. **Positional embeddings** - Adding position information

## Install Dependencies

In [2]:
import requests

url = "https://www.gutenberg.org/cache/epub/84/pg84.txt"
filename = "pg84.txt"

try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"Successfully downloaded '{filename}'")
except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")

Successfully downloaded 'pg84.txt'


## Download Training Data

We download "Frankenstein" by Mary Shelley from Project Gutenberg — a free source of public domain books. This text will be used to train our tokenizer and create training examples.

In [3]:
with open("pg84.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

## Load the Text into Memory

Read the entire text file into a string variable. This raw text will be tokenized and used to create training examples.

## Create a Custom PyTorch Dataset

**GPTDatasetV1** creates training examples using a **sliding window** approach for next-token prediction.

**Key concept:** For language modeling, the target is always the **next token** after each input token.

```
Input:  [token_0, token_1, token_2, token_3]
Target: [token_1, token_2, token_3, token_4]
```

**Parameters:**
- `max_length`: Size of each training sequence (context window)
- `stride`: How many tokens to slide between sequences
  - `stride = 1`: Maximum overlap, more training examples
  - `stride = max_length`: No overlap, non-overlapping chunks

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

# Define a custom Dataset for the text data

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

## Create DataLoader Helper Function

This function wraps the dataset creation and returns a PyTorch DataLoader.

Uses **tiktoken** with GPT-2's **BPE (Byte Pair Encoding)** tokenizer — the same tokenizer used by OpenAI's GPT-2 model.

In [5]:
import tiktoken

def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

## Test the DataLoader: Sliding Window with stride=1

With `stride=1`, consecutive batches overlap by all but one token. Notice how:
- First batch input: `[171, 119, 123, 464]`
- Second batch input: `[119, 123, 464, 4935]` — shifted by 1 token

This creates maximum training examples but with lots of redundancy.

In [6]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[171, 119, 123, 464]]), tensor([[ 119,  123,  464, 4935]])]


In [7]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 119,  123,  464, 4935]]), tensor([[  123,   464,  4935, 20336]])]


## Batch Processing with stride=max_length (No Overlap)

With `stride=4` (equal to `max_length`), we get **non-overlapping chunks**. This is more memory-efficient for training.

Notice:
- Each row in **Inputs** is a separate training sequence
- Each row in **Targets** is shifted by 1 position (the next tokens)
- Shape is `[batch_size=8, max_length=4]`

In [ ]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  171,   119,   123,   464],
        [ 4935, 20336, 46566,   286],
        [45738,    26,  1471,    11],
        [  383, 12495, 42696,   198],
        [  220,   220,   220,   220],
        [  198,  1212, 47179,   318],
        [  329,   262,   779,   286],
        [ 2687,  6609,   287,   262]])

Targets:
 tensor([[  119,   123,   464,  4935],
        [20336, 46566,   286, 45738],
        [   26,  1471,    11,   383],
        [12495, 42696,   198,   220],
        [  220,   220,   220,   198],
        [ 1212, 47179,   318,   329],
        [  262,   779,   286,  2687],
        [ 6609,   287,   262,  1578]])


## BPE Tokenization Statistics

**BPE (Byte Pair Encoding)** is a subword tokenization algorithm:
- Starts with individual characters
- Iteratively merges the most frequent pairs
- Results in a vocabulary of common subwords

GPT-2's vocabulary size is **50,257** tokens. The entire "Frankenstein" book tokenizes to ~106k tokens.

In [8]:
import tiktoken

# Get the gpt2 tokenizer
tokenizer_bpe = tiktoken.get_encoding("gpt2")

# Encode the raw_text
tokens_bpe = tokenizer_bpe.encode(raw_text)

# Print the number of tokens
print(f"Total number of tokens in raw_text using GPT-2 BPE encoding: {len(tokens_bpe)}")

Total number of tokens in raw_text using GPT-2 BPE encoding: 106361


## Create Token Embedding Layer

An **embedding layer** is a lookup table that maps each token ID to a dense vector.

- `vocab_size = 50,257`: Number of possible tokens (rows in the table)
- `output_dim = 256`: Size of each embedding vector (columns)

The embedding weights are **learnable parameters** — they start random and are updated during training to capture semantic meaning.

In [9]:
vocab_size = tokenizer_bpe.n_vocab
print(vocab_size)
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

50257


In [10]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
   stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[  171,   119,   123,   464],
        [ 4935, 20336, 46566,   286],
        [45738,    26,  1471,    11],
        [  383, 12495, 42696,   198],
        [  220,   220,   220,   220],
        [  198,  1212, 47179,   318],
        [  329,   262,   779,   286],
        [ 2687,  6609,   287,   262]])

Inputs shape:
 torch.Size([8, 4])


## Apply Token Embeddings

Pass token IDs through the embedding layer to get dense vectors.

**Shape transformation:**
- Input: `[8, 4]` → 8 sequences, each with 4 token IDs
- Output: `[8, 4, 256]` → each token ID is now a 256-dimensional vector

This is a simple **lookup operation** — each token ID retrieves its corresponding row from the embedding table.

In [11]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


## Create Positional Embeddings

**Problem:** Token embeddings alone don't encode **position** information. "The cat sat on the mat" and "mat the on sat cat the" would have the same bag of embeddings!

**Solution:** Add **positional embeddings** — a unique vector for each position in the sequence.

- `context_length = 4`: Maximum sequence length
- Each position (0, 1, 2, 3) gets its own 256-dimensional embedding
- Shape: `[4, 256]` — one embedding per position

These are also **learnable parameters** in GPT-2 (unlike the fixed sinusoidal embeddings in the original Transformer).

In [12]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


## Combine Token + Positional Embeddings

The final input to the transformer is the **sum** of token embeddings and positional embeddings:

```
input_embeddings = token_embeddings + pos_embeddings
```

**Broadcasting:** The positional embeddings `[4, 256]` are automatically broadcast across all 8 sequences, so each sequence gets the same positional information added.

**Final shape:** `[8, 4, 256]` — ready to feed into the transformer layers!

This is the complete embedding pipeline:
1. Text → Token IDs (tokenization)
2. Token IDs → Token Embeddings (lookup)
3. Token Embeddings + Positional Embeddings → Input Embeddings

In [13]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
